In [2]:
import tensorflow as tf
from tensorflow import keras

## Tensorflow 특징

- 넘파이와 비슷하지만 GPU 지원, TPU 또한


- 분산 컴퓨팅 지원


- 자동 미분 및 고성능 옵티마이저 제공


- 텐서플로 허브, https://github.com/tensorflow/models 에서 많은 모델들을 다운 받을 수 있다.

## 12.2 넘파이 처럼 텐서플로 사용하기

### 12.2.1 텐서와 연산

In [3]:
tf.constant([[1,2,3],[4,5,6]])

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [4]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1,2,3],[4,5,6]])
t.shape

TensorShape([2, 3])

In [6]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [5, 6]])>

In [7]:
t

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [8]:
t[...,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [5]])>

In [9]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[11, 12, 13],
       [14, 15, 16]])>

In [10]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 1,  4,  9],
       [16, 25, 36]])>

In [11]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

### 12.2.2 텐서와 넘파이

In [12]:
import numpy as np
a = np.array([2,4,5])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 5])>

In [13]:
t.numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>>

In [14]:
np.square(t) # tensor를 array 로 변환

array([[ 1,  4,  9],
       [16, 25, 36]], dtype=int32)

### 12.2.3 타입 변환

- 타입 변환은 선능을 크게 감소시킨다.
- tensorflow는 어떤 타입 변환도 자동으로 수행하지 않는다.

In [15]:
tf.constant(2.) + tf.constant(40) # 정수, 실수간 연산은 안됩니다.
tf.constant(2.) + tf.constant(40.,dtype=tf.float64) # 이것도 안됩니다.

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [16]:
t2 = tf.constant(40.,dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32) # 타입 변환 함수 cast

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### 12.2.4 변수
- 일반적인 tensor 객체는 변경이 불가능하다.

In [17]:
v = tf.Variable([[1,2,3],[4,5,6]]) # 일반 tensor 처럼 연산을 할 수 있다.
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [18]:
# assign 함수를 이용하여 값 변경하기

v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 10, 12]])>

In [19]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2, 42,  6],
       [ 8, 10, 12]])>

In [20]:
v[:,2].assign([0,1]) # 입력시 dtype에 주의하자!! [0.,1.]은 불가

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2, 42,  0],
       [ 8, 10,  1]])>

### 12.2.5 다른 데이터 구조      

- 희소 텐서(tf.SparseTensor) : 대부분 0으로 채워진 텐서를 효율적으로 나타낸다.  


- 텐서 배열(tf.TensorArray) : 텐서 리스트, 같은 크기, 타입의 텐서만 있어야한다.


- 문자열 텐서, 집합, 큐 등이 있다.(필요하면 책 찾아보기)

## 12.3 사용자 정의 모델과 손실함수

### 12.3.1 사용자 정의 손실 함수

- 후버 손실함수(huber)

In [21]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)
'''
성능을 위해서 벡터화, 텐서플로 그래프의 장점을 활용하기 위해 텐서플로 연산만을 이용한다.
전체 데이터의 평균이 아닌 각각의 값을 반환하는 것이 좋다. 클래스 또는 샘플 가중치를 이용하기 위해
'''

'\n성능을 위해서 벡터화, 텐서플로 그래프의 장점을 활용하기 위해 텐서플로 연산만을 이용한다.\n전체 데이터의 평균이 아닌 각각의 값을 반환하는 것이 좋다. 클래스 또는 샘플 가중치를 이용하기 위해\n'

In [22]:
'''캘리포니아 주택가격 데이터'''

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [23]:
input_shape = X_train.shape[1:]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

In [24]:
# model = tf.keras.models.load_model('./my_model_A.h5')
model.compile(loss=huber_fn, optimizer='nadam',metrics=['mae'])
model.fit(X_train, y_train, epochs=5,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
363/363 [==============================] - 3s 7ms/step - loss: 6.0557 - mae: 6.5309 - val_loss: 1.7789 - val_mae: 2.2362
Epoch 2/5
363/363 [==============================] - 2s 4ms/step - loss: 2.9029 - mae: 3.3731 - val_loss: 1.7552 - val_mae: 2.2143
Epoch 3/5
363/363 [==============================] - 2s 5ms/step - loss: 2.6361 - mae: 3.1039 - val_loss: 1.7387 - val_mae: 2.1994oss: 2.6525 - mae
Epoch 4/5
363/363 [==============================] - 2s 4ms/step - loss: 2.5716 - mae: 3.0393 - val_loss: 1.7242 - val_mae: 2.1871
Epoch 5/5
363/363 [==============================] - 2s 4ms/step - loss: 2.2714 - mae: 2.7320 - val_loss: 1.7170 - val_mae: 2.1805


### 사용자 정의 요소를 가진 모델을 저장하고 로드하기

In [25]:
model.save('model_12_huber_fn.h5')

In [26]:
model = tf.keras.models.load_model('./model_12_huber_fn.h5', custom_objects={'huber_fn':huber_fn})
''' 사용자 정의 요소의 이름을 그대로 저장 -> 모델 load시 모델에 있는 이름과 객체를 매핑한다.'''

' 사용자 정의 요소의 이름을 그대로 저장 -> 모델 load시 모델에 있는 이름과 객체를 매핑한다.'

In [27]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error)/2
        linear_loss = 2 * threshold * tf.abs(error) - threshold ** 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0),optimizer='nadam')

# 모델을 로드할 때 threshold 값을 새로 입력해 주어야한다.
# model = tf.keras.models.load_model('./model_12_huber_fn.h5', custom_objects={'huber_fn':create_huber(2.0)})

In [28]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_ture,y_pred):
        error = tf.abs(y_true-y_pred)
        is_small_error = error < self.threshold
        squared_error = error **2
        linear_error = 2 * self.threshold * error - self.threshold ** 2
        return tf.where(is_small_error,squared_error,linear_error)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold':self.threshold}
    
# get_config 함수로 threshold 까지 저장이 가능하다.
# model.compile(loss=HuberLoss(2.0),optimizer='nadam')
# model = tf.keras.models.load_model('./model_12_huber_fn.h5', custom_objects={'HuberLoss':HuberLoss})

### 활성화 함수, 초기화, 규제, 제한을 커스터마이징 하기

In [29]:
# 사용자 정의로 각각을 만들어 사용해보기

def my_softpuls(z):
    return tf.math.log(tf.exp(z) +1.0)

def my_glorot_initializer(shape, detype=tf.float32):
    stddv = tf.sqrt(2./(shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev,dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))

def my_positive_weights(weights):
    return tf.where(weight<0.,tf.zeros_like(weights),weights)

In [30]:
layer = tf.keras.layers.Dense(30,activation=my_softpuls
                             ,kernel_initializer=my_glorot_initializer
                             ,kernel_regularizer=my_l1_regularizer
                             ,kernel_constraint=my_positive_weights)

In [31]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self,factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {'factor':self.factor}

### 사용자 정의 지표



- 손실과 지표: 큰 차이는 아니다. 둘 다 모델을 평가하기 위해 사용한다.


- 손실은 모델을 훈련하기 위함이므로 미분 가능, 그레디언트 등 수학적으로 고려해서 정해야한다.


- 지표는 사람이 모델을 평가할 때 사용하기 때문에 직관적으로 이해가 쉬워야 한다.

- 스트리밍 지표 : 배치마다 지표의 단순 평균을 내는 것이 아니라 총 배치의 결과를 바탕으로 계산하는 방식



    - ex. 정밀도를 지표로 사용 할 때  
        
    배치 1. 예측 양성: 5, 진짜 양성: 4, 정밀도 80%  
       
    배치 2. 예측 양성: 3, 진짜 양성: 0, 정밀도 0%  
        
    단순 평균으로 계산한 절밀도 지표 = (80 + 0)/2 = 40%  
    
    전체 결과를 바탕으로 계산한 정밀도 지표 = (4+0)/(5+3) = 50%  

In [32]:
# 정밀도를 스트리밍 지표로 사용하기

precision = tf.keras.metrics.Precision() # 스트리밍 지표
precision([0,1,1,1,0,1,0,1],[1,1,0,1,0,1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [33]:
precision([0,1,0,0,1,0,1,1],[1,0,1,1,0,0,0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [34]:
precision.result() # numpy = 값 -> 현재 정밀도 (배치별 평균을 내는 것이 아니라 모든 배치의 결과를 저장한 뒤 계산)

precision.variables # 변수확인

precision.reset_states() # 변수를 0.0으로 바꾼다.

In [35]:
# 스트리밍 지표로 후버 함수 만들기
# keras.metrics.Metric을 상속

In [36]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # 기본 매개변수 처리
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight('total', initializer='zeros')
        self.count = self.add_weight('count', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true,y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        return self.total/self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

### 사용자 정의 층

In [37]:
# 가중치가 없는 층(연산, 계수가 없는 층)
# keras.layers.Lambda 를 상속를 상속
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))

In [38]:
# 가중치가 있는 층
# keras.layers.Layer 상속

class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name='kernel', shape=[batch_input_shape[-1], self.unit]
                                     ,initializer = 'glorot_normal')
        self.bias = self.add_weight(name='bias',shape=[self.units],initializer='zeros')
        super().build(batch_input_shape) # 마지막에 호출해야 한다.
        
    def call(self,X):
        return self.activation(X @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1]+ [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units':self.units,'activation':tf.keras.activations.serialize(self.activation)}

In [39]:
# 2개의 입력과 3개의 출력을 갖을 때

class MyMultilayer(tf.keras.layers.Layer):
    def call(self,X):
        X1,X2 = X
        return [X1 + X2, X1 * X2, X1/X2]
    
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1,b1,b1]

In [40]:
# 훈련과 테스트에서 다르게 작용하는 층

class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    
    def call(self,X,training=None): # training option을 부여한다.
        if training:
            noise = tf.random.normal(tf.shape(X),stddev=self.stddev) #훈련중에만 가우시안 잡음을 추가
            return X + noise
        else:
            return X
    
    def compute_output_shape(self, batch_input_shape):
        return bacth_input_shape

### 사용자 정의 모델

- 스킵연결이 있는 residual block으로 이루어진 모델 제작해보기

In [41]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layer, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation='elu',kernel_iniitalizer='he_normal') 
                       for _ in range(n_layer)]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [42]:
class ResidualRegressor(keras.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30,activation='elu',kernel_initializer='he_normal')
        self.block1 = ResidualBlock(2,30)
        self.block1 = ResidualBlock(2,30)
        self.out = keras.layers.Dense(output_dim)
    
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1+3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

### 모델 구성 요소에 기반한 손실과 지표

In [43]:
class RecondtructingRegressor(keras.Model):
    def __init__(self,output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation='selu',kernel_iniitalizer='lecun_normal') for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        
    def build(self,bacth_input_shape):
        n_input = bacth_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(bacth_input_shape)
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05*recon_loss) # 오류에 reconstruct loss 더하기 
        return self.out(Z)

### 자동 미분을 사용하여 그레디언트 계산하기

- 여러 값에 대한 한 값의 그레디언트를 계산 할 때 사용된다.


- ex. 모델 파라미터로 계산된 손실 함수 값


- 정방향 계산과 역방향 계산을 동시에 수행할 수 있다.




In [44]:
def f(w1, w2):
    return 3*w1**2 + 2*w1*w2

In [45]:
# 도함수의 근삿값을 계산

w1, w2 = 5,3
eps = 1e-6

display((f(w1+eps,w2) - f(w1,w2))/eps)
display((f(w1,eps+w2) - f(w1,w2))/eps)

36.000003007075065

10.000000003174137

In [46]:
# 자동 미분
w1,w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1,w2)

gradients = tape.gradient(z,[w1,w2]) # 호출 후에 테이프는 사라진다. 2번 호출하면 에러난다.
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [47]:
with tf.GradientTape(persistent=True) as tape: # 여러번 사용하는 방법
    z = f(w1,w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)
del tape # 꼭 삭제를 진행하여 리소스를 해제해야 한다.

dz_dw1, dz_dw2

(<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>)

In [49]:
# 기본적으로 테이프는 변수만 연산에 포함시킨다.
# 강제로 시킬 수는 있다.

c1,c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1,c2)

print('Non Variable')
display(tape.gradient(z,[c1,c2]))

with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1,c2)

print('Variable')
display(tape.gradient(z,[c1,c2]))


Non Variable


[None, None]

Variable


[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [51]:
# 일부분의 그레디언트가 역전파되지 않도록 막는 방법, 정방향 계산은 됩니다.

def f(w1,w2):
    return 3*w1**2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
    z = f(w1,w2)

gradients = tape.gradient(z,[w1,w2])

gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

In [52]:
# 큰값에 대한 자동 미분은 불안정하여 계산되지 않는다.
x = tf.Variable([100.])
with tf.GradientTape() as tape:
    z = my_softpuls(x)

tape.gradient(z,[x])

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([nan], dtype=float32)>]

In [53]:
@tf.custom_gradient
def my_better_softplus(z):
    exp = tf.exp(z)
    def my_softplus_gradients(grad):
        return grad / (1+1/exp)
    return tf.math.log(exp+1), my_softplus_gradients

### 사용자 정의 훈련 반복

In [54]:
# 모델 만들기
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([keras.layers.Dense(30,activation='elu',kernel_initializer='he_normal'
                                                   ,kernel_regularizer=l2_reg)
                                ,keras.layers.Dense(1,kernel_regularizer=l2_reg)])

In [65]:
# 배치를 랜덤하게 추출하는 함수
def random_batch(X,y,batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [69]:
# 훈련상태를 출력하는 함수
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = ' - '.join(['{}: {:.4f}'.format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = '' if iteration < total else '\n'
    print('\r{}/{} - '.format(iteration, total) + metrics, end = end)

In [62]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [93]:
# 훈련

for epoch in range(1, n_epochs+1):
    print('epoch {}/{}'.format(epoch,n_epochs))
    for step in range(1, n_steps+1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch,y_pred))
            loss = tf.add_n([main_loss]+model.losses)
        gradients = tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step*batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

epoch 1/5
11610/11610 - mean: 0.6210 - mean_absolute_error: 0.5030
epoch 2/5
11610/11610 - mean: 0.6304 - mean_absolute_error: 0.5120
epoch 3/5
11610/11610 - mean: 0.6391 - mean_absolute_error: 0.5136
epoch 4/5
11610/11610 - mean: 0.6278 - mean_absolute_error: 0.5045
epoch 5/5
11610/11610 - mean: 0.6319 - mean_absolute_error: 0.5106


## 텐서플로 함수와 그래프

In [101]:
def cube(x):
    return x **3

In [103]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [104]:
tf_cube = tf.function(cube)
tf_cube

In [105]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [106]:
@tf.function
def tf_cube(x):
    return x**3

In [107]:
tf_cube.python_function(4)

64

In [112]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [113]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>